[Tutorial URL](https://python.langchain.com/docs/get_started/quickstart)

In [1]:
# Install the required packages

!pipenv install langchain-openai
!pipenv install beautifulsoup4
!pipenv install langchain-community
!pipenv install faiss-cpu
!pipenv install langchain

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langchain-openai...
Resolving langchain-openai...
✔ Installation Succeeded
⠋ Installing langchain-openai...
Installing dependencies from Pipfile.lock (884138)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set 

In [2]:
# type: ignore
# get the OPENAI_API_KEY from the .env file
OPENAI_API_KEY = %env OPENAI_API_KEY
# get the LANGCHAIN_API_KEY from the .env file
LANGCHAIN_API_KEY = %env LANGCHAIN_API_KEY
LANGCHAIN_TRACING_V2 = %env LANGCHAIN_TRACING_V2
TAVILY_API_KEY = %env TAVILY_API_KEY


In [3]:
from langchain_openai import ChatOpenAI
# type: ignore
llm = ChatOpenAI()

In [17]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing in the following ways:\n\n1. Test Automation: Langsmith can automate the testing process by generating test cases and test scripts based on the provided specifications. This saves time and effort in manually writing test cases and allows for faster and more efficient testing.\n\n2. Test Coverage Analysis: Langsmith can analyze the codebase and provide insights into the areas of the code that need more testing coverage. It can identify potential gaps in the testing and suggest additional test cases to ensure comprehensive coverage.\n\n3. Test Scenario Generation: Langsmith can generate various test scenarios based on the given input parameters and system specifications. It can simulate different scenarios to test the system's behavior and identify any potential issues or bugs.\n\n4. Error Detection: Langsmith can help in detecting errors by analyzing the code and identifying potential bugs, vulnerabilities, or performance issues. It can

>We can also guide it's response with a prompt template. Prompt templates are used to convert raw user input to a better input to the LLM.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are world class technical documentation writer.'),
    ('user', '{input}')
])


>We can now combine these into a simple LLM chain:

In [5]:
chain = prompt | llm

>We can now invoke it and ask the same question. It still won't know the answer, but it should respond in a more proper tone for a technical writer!

In [6]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith is a powerful tool that can greatly assist with testing in several ways. Here are a few ways Langsmith can be beneficial for testing purposes:\n\n1. Test Case Generation: Langsmith can automatically generate test cases based on various inputs, such as requirements, specifications, or even existing code. It can analyze the codebase and generate a set of test cases that cover different execution paths, boundary conditions, and possible edge cases. This can save significant time and effort in manually creating test cases.\n\n2. Test Execution: Langsmith can execute test cases and provide detailed reports on the test results. It can automatically run the generated test cases and identify any failures or errors encountered during the execution. This helps in quickly identifying and resolving issues, improving the efficiency of the testing process.\n\n3. Test Coverage Analysis: Langsmith can analyze the test coverage by mapping the executed code paths against the

>The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

> Now add to the previous chain

In [8]:
chain = prompt | llm | output_parser

We can now invoke it and ask the same question. The answer will now be a string (rather than a ChatMessage).

In [23]:
chain.invoke({"input": "how can langsmith help with testing?"})

"Langsmith can help with testing in several ways:\n\n1. Test Case Creation: Langsmith can assist in creating test cases by generating test inputs and expected outputs based on the specifications of the system under test. This can save time and effort in manually writing test cases.\n\n2. Test Data Generation: Langsmith can automatically generate test data for different scenarios, including edge cases and boundary conditions. This ensures comprehensive testing coverage and helps identify potential issues that might arise with specific inputs.\n\n3. Test Execution: Langsmith can execute test cases automatically, reducing the need for manual testing. This can be especially useful for repetitive or time-consuming test scenarios, allowing testers to focus on more complex and critical areas.\n\n4. Test Result Comparison: After test execution, Langsmith can compare the actual outputs with the expected outputs to identify any discrepancies. This speeds up the process of analyzing test results 

### Retrieval Chain

> In order to properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via retrieval. Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

> In this process, we will look up relevant documents from a Retriever and then pass them into the prompt. A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. For more information on vectorstores, see this documentation.

>First, we need to load the data that we want to index. In order to do this, we will use the WebBaseLoader. This requires installing BeautifulSoup:

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

>Next, we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.
For embedding models, we once again provide examples for accessing via OpenAI or via local models.

>Make sure you have the `langchain_openai` package installed an the appropriate environment variables set (these are the same as needed for the LLM).

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


>Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, FAISS, for simplicity's sake.
First we need to install the required packages for that: `pip install faiss-cpu`
Then we can build our index:

In [11]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
vector

>Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.
First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

>If we wanted to, we could run this ourselves by passing in documents directly:

In [13]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

>However, we want the documents to first come from the retriever we just set up. That way, for a given question we can use the retriever to dynamically select the most relevant documents and pass those in.

In [14]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response['answer'])

LangSmith can help with testing by providing a way to construct datasets, run chains and agents over the data points, and visualize the outputs. It allows users to easily upload datasets and run chains over them, logging the results to a new project associated with the dataset. The results can be reviewed, and feedback can be assigned to runs. LangSmith also provides evaluators that can be specified during a test run to evaluate the results. Additionally, LangSmith offers annotation queues for manual review and annotation of runs, allowing users to assess subjective qualities and validate auto-evaluated runs.


## Conversation Retrieval Chain

> The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

> We can still use the create_retrieval_chain function, but we need to change two things:

>The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
The final LLM chain should likewise take the whole history into account

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search Query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", '{input}'),
    ("user", "Given the above conversation, generate a search query to lookup in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

>We can test this out by passing in an instance where the user is asking a follow up question.

In [17]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past

>You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow up question.
Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", '{input}')
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [19]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes')],
 'input': 'Tell me how',
 'context': [Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
  Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is 

## Agent

We've so far create examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

**NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.=**

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access two tools:
1. The retriever we just created. This will let it easily answer questions about LangSmith
1. A search tool. This will let it easily answer questions that require up to date information.

First, let's set up a tool for the retriever we just created:

### Setup

In [20]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [21]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

We can now create a list of the tools we want to work with:

In [22]:
tools = [retriever_tool, search]

Now that we have the tools, we can create an agent to use them. We will go over this pretty quickly - for a deeper dive into what exactly is going on, check out the Agent's Getting Started documentation

Install langchain hub first

In [23]:
!pipenv install langchainhub

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langchainhub...
Resolving langchainhub...
Added langchainhub to Pipfile's [packages] ...
✔ Installation Succeededainhub...
⠋ Installing langchainhub...
Pipfile.lock (884138) out of date, updating to (aaab0e)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠸ Locking...
Locking [dev-packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠧ Locking...
Updated Pipfile.lock (29d0d06fca062ee2bbf608e99036fd6568d0f1c02e0704385a8803521faaab0e)!
Installing dependencies from Pipfile.lock (aaab0e)...
To activate thi

Now we can use it to get a predefined prompt

In [24]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith:

In [25]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the exec

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.\n\n4. Test Prioritization: Lan

We can ask it about the weather:

In [26]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/february/', 'content': 'Best time to go to San Francisco? Weather in San Francisco in february 2024  How was the weather last february? Here is the day by day recorded weather in San Francisco in february 2023:  Seasonal average climate and temperature of San Francisco in february  The climate of San Francisco in february is goodSan Francisco at the Beginning of February ... Temperatures ranging from 9° C to 14° C translate into very mild mornings and pleasant days. However, chilly people\xa0...'}, {'url': 'https://www.metcheck.com/WEATHER/dayforecast.asp?location=San%20Francisco&locationID=414573&lat=8.870940&lon=-82.823250&dateFor=12/02/2024', 'content': 'Weather Forecast for\xa0San Francisco\xa0for\xa0Monday 12 February Model/Location Information Help & Information  Showing

{'input': 'what is the weather in SF?',
 'output': 'The weather in San Francisco is currently not available. However, you can check the weather forecast for San Francisco on this [website](https://www.metcheck.com/WEATHER/dayforecast.asp?location=San%20Francisco&locationID=414573&lat=8.870940&lon=-82.823250&dateFor=12/02/2024).'}

We can have conversations with it:

In [27]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, and overall impact.

2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and effectively presents your qualifications.

3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders and offer suggestions on what information to include in your recommendation letter requests.

4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting the right law schools, organizing your application materials, and managing deadlines.

5. Interview Preparation: LangSmith can prov

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, and overall impact.\n\n2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and effectively presents your qualifications.\n\n3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders and offer suggestions on what information to include in your recommendation letter requests.\n\n4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting the righ

## Serving with LangServe

Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

In [28]:
!pipenv install "langserve[all]"

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langserve...
Resolving langserve[all]...
Added langserve to Pipfile's [packages] ...
✔ Installation Succeededrve...
⠋ Installing langserve...
Pipfile.lock (aaab0e) out of date, updating to (31a871)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠋ Locking...
Locking [dev-packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠙ Locking...
Updated Pipfile.lock (b9c0d4601b96dbb3f763772ea66b10c1511d812a08d833611f6e13789531a871)!
Installing dependencies from Pipfile.lock (31a871)...
To activate this project'

### Server

To create a server for our application we'll make a serve.py file. This will contain our logic for serving our application. It consists of three things:
1. The definition of our chain that we just built above
1. Our FastAPI app
1. A definition of a route from which to serve the chain, which is done with langserve.add_routes

serve.py
```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes

# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )


class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

And that's it! If we execute this file:
`python serve.py`
we should see our chain being served at localhost:8000.

### Playground
Every LangServe service comes with a simple built-in UI for configuring and invoking the application with streaming output and visibility into intermediate steps. Head to http://localhost:8000/agent/playground/ to try it out! Pass in the same question as before - "how can langsmith help with testing?" - and it should respond same as before.

### Client
Now let's set up a client for programmatically interacting with our service. We can easily do this with the [langserve.RemoteRunnable](/docs/langserve#client). Using this, we can interact with the served chain as if it were running client-side.

In [29]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke({
    "input": "how can langsmith help with testing?",
    "chat_history": []  # Providing an empty list as this is the first call
})

{'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been tested and may require additional test cases.\n\n4. Test Prioritization: LangSmith can prioritize the execution of test cases based on th

### Next steps

We've touched on how to build an application with LangChain, how to trace it with LangSmith, and how to serve it with LangServe. There are a lot more features in all three of these than we can cover here. To continue on your journey, we recommend you read the following (in order):

1. All of these features are backed by LangChain Expression Language (LCEL) - a way to chain these components together. Check out that documentation to better understand how to create custom chains.
1. Model IO covers more details of prompts, LLMs, and output parsers.
1. Retrieval covers more details of everything related to retrieval
1. Agents covers details of everything related to agents
1. Explore common end-to-end use cases and template applications
1. Read up on LangSmith, the platform for debugging, testing, monitoring and more
1. Learn more about serving your applications with LangServe